In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
# List all physical devices recognized by TensorFlow
physical_devices = tf.config.list_physical_devices('GPU')
print("Available GPUs:", physical_devices)
if gpus :
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 4)])

    except RuntimeError as e:
        print(e)

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

# **Uploading file from your directory**
 The file you upload should be a zip folder containing your dataset.
This code below will prompt you to select and upload the zip file from your local computer into colab directory

In [ ]:
#comment out cell if using a local machine

from google.colab import files
uploaded = files.upload()


# **Unziping the Folder in Colab**
Here you should be able to see the extracted zip folder in your colab folder directory

In [ ]:
#comment out cell if using a local machine

import zipfile

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        zip_ref = zipfile.ZipFile(filename, 'r')
        zip_ref.extractall()
        print(f"Extracted folder correctly {filename}")
        zip_ref.close()


# **Import Libraries**

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

##  **Preparing and getting the file directories**
Here we are getting the folder directories for the Car and Bike folder which is found in the parent folder called Car-Bike-Dataset which we have already extracted above from our zip folder if we used Colab environment. With your local machine it should open the current directory where your dataset is located and point to it.

In [ ]:
# if using local machine, ensure that the zip file has been unzipped in the same
#folder as the one which has this notebook

# Get the current directory
current_dir = os.getcwd()
print("Current directory:", current_dir)

# Set the correct target folder path
target_folder = os.path.join(current_dir, "Car-Bike-Dataset")
print("Target folder path:", target_folder)



## Getting the path for the training and test images for the Car and bike

In [ ]:
# Define the actual train and test paths for both Car and Bike
car_train_folder = os.path.join(target_folder, "Car", "train_images")
car_test_folder = os.path.join(target_folder, "Car", "test_images")
bike_train_folder = os.path.join(target_folder, "Bike", "train_images")
bike_test_folder = os.path.join(target_folder, "Bike", "test_images")

print("Car train data path:", car_train_folder)
print("Car test data path:", car_test_folder)
print("Bike train data path:", bike_train_folder)
print("Bike test data path:", bike_test_folder)

In [ ]:
IMAGE_SIZE = 150

# Getting and loading the dataset
Here, this function below will help us load the dataset we obatined above for the car_folder and bike_folder respectively

In [ ]:
# Function to load dataset
def load_dataset(folder, label):
    data = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            image = load_img(img_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
            image = img_to_array(image) / 255.0
            data.append(image)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {img_path}: {e}")

    return np.array(data), np.array(labels)

## Load the data

In [ ]:
# Load training data
train_images_car, train_labels_car = load_dataset(car_train_folder, 1)
train_images_bike, train_labels_bike = load_dataset(bike_train_folder, 0)

In [ ]:
# Load testing data
test_images_car, test_labels_car = load_dataset(car_test_folder, 1)
test_images_bike, test_labels_bike = load_dataset(bike_test_folder, 0)

In [ ]:
# Combine training data
train_images = np.concatenate([train_images_car, train_images_bike])
train_labels = np.concatenate([train_labels_car, train_labels_bike])

# Combine testing data
test_images = np.concatenate([test_images_car, test_images_bike])
test_labels = np.concatenate([test_labels_car, test_labels_bike])

## Check the number of images loaded for training and testing

In [ ]:
print(f"Number of training images: {len(train_images)}")
print(f"Number of test images: {len(test_images)}")

## Displaying sample images for Car and Bike

In [ ]:
# Function to plot 5 sample images
def plot_sample_images(images, labels, num_samples=10, images_per_row=5):
    num_rows = (num_samples + images_per_row - 1) // images_per_row
    plt.figure(figsize=(images_per_row * 3, num_rows * 3))

    indices = random.sample(range(len(images)), num_samples)

    for i, index in enumerate(indices):
        plt.subplot(num_rows, images_per_row, i + 1)
        plt.imshow(images[index])
        label = 'Car' if labels[index] == 1 else 'Bike'
        plt.title(label)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Plot 10 sample images (5 images per row)
plot_sample_images(train_images, train_labels, num_samples=10, images_per_row=5)


<!-- ## Split the data into train and test sets
To help our model learn correctly we are splitting the dataset into 90% training and 10% testing -->

# Define your model for training

In [ ]:
#Build your model
model = tf.keras.models.Sequential([
   #Insert your model here
])
model.summary()

# Compiling the model

In [ ]:
#Feel free to update optimizer

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

##  Train the model

In [ ]:
#Feel free to update hyperparameters

history = model.fit(
    train_images, train_labels,
    epochs=20,
    batch_size=32,
    validation_split=0.1
)

## Evaluating the model
Here we evaluate the model on the test set to see the final accuracy

In [ ]:
# Evaluate the model on the test set (validation set)

test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Set Accuracy: {test_accuracy * 100:.2f}%")


## Visualization of Training and Validation Loss and Accuracy
The plots provide a visual representation of the model's training process over multiple epochs.

In [ ]:

plt.figure(figsize=(12, 5))

# Plot training & validation loss values
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training & validation accuracy values
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


## Getting to see the predicted images for both car and bike
Here, based on how well the model was able to train you will be able to see if predicting 90% accurately from the test images

In [ ]:
# Function to predict and visualize multiple images from the test set
def predict_and_visualize_multiple(model, test_images, test_labels, num_samples=10, images_per_row=5):
    num_rows = (num_samples + images_per_row - 1) // images_per_row
    plt.figure(figsize=(images_per_row * 3, num_rows * 3))

    # Randomly sample indices from the test set
    indices = random.sample(range(len(test_images)), num_samples)

    for i, index in enumerate(indices):
        plt.subplot(num_rows, images_per_row, i + 1)
        img = test_images[index]
        true_label = 'Car' if test_labels[index] == 1 else 'Bike'

        # Expand the dimensions for model prediction
        img_array = np.expand_dims(img, axis=0)

        # Predict the label
        prediction = model.predict(img_array)
        predicted_label = 'Car' if prediction[0][0] > 0.5 else 'Bike'
        probability = prediction[0][0]

        # Display the image and the prediction
        plt.imshow(img)
        plt.title(f"True: {true_label}\nPred: {predicted_label} ({probability:.2f})")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Call the function to predict and visualize 10 random images from the test set
predict_and_visualize_multiple(model, test_images, test_labels, num_samples=10, images_per_row=5)

